In [9]:
import json
from pprint import pprint
import secrets as secrets_module

from dydx3 import Client
from dydx3 import private_key_to_public_key_pair_hex
from dydx3.constants import NETWORK_ID_ROPSTEN, NETWORK_ID_MAINNET
from dydx3.constants import API_HOST_ROPSTEN, API_HOST_MAINNET
from web3 import Web3

def generate_private_key():
    priv = secrets_module.token_hex(32)
    private_key = "0x" + priv
    return private_key

def onboard_initial_account(eth_private_key, network_id, host):
    print(f"Eth private key: {eth_private_key}\n")

    dydx_client = Client(
        network_id=network_id,
        host=host,
        eth_private_key=eth_private_key,
    )
    
    stark_private_key = dydx_client.onboarding.derive_stark_key()
    dydx_client.stark_private_key = stark_private_key
    print(f"STARK private key: {stark_private_key}\n")
    
    public_x, public_y = private_key_to_public_key_pair_hex(stark_private_key)
    print(f"STARK (public x, public y): ({public_x},{public_y})\n")
    
    onboarding_response = dydx_client.onboarding.create_user(
        stark_public_key=public_x,
        stark_public_key_y_coordinate=public_y,
    ).data
    print("Account onboarding response:")
    pprint(onboarding_response)
    return

secrets = {}
# ensure that the secrests.json file contains `infura_ropsten_endpoint` attribute
with open("secrets.json") as secretfile:
    secrets = json.load(secretfile)

# ropsten check (with randomly generated ethereum address)
onboard_initial_account(
    generate_private_key(),
    NETWORK_ID_ROPSTEN,
    API_HOST_ROPSTEN
)

# # mainnet check
# onboard_initial_account(
#     secrets["eth_private_endpoint"], # must provide a used account's private key, else get `DydxApiError(status_code=400, response={'errors': [{'msg': 'User wallet has no transactions, Ethereum or USDC'}]})`
#     secrets["infura_mainnet_key"],
#     NETWORK_ID_MAINNET,
#     API_HOST_MAINNET
# )

Eth private key: 0x69d99d047508db8732e04f312c15b64860f6f79374ff035ec1c607be71d8d8d3

STARK private key: 0x7faedf02e5017a0973eac623b1814e936d109ec842f6e109ec5610493c1e82b

STARK (public x, public y): (0x2e0e562caf451a1c91f9adc57117d6e1a00dd039f6990e8db2b394584db6bdf,0x69362234c3fe2acf26f0aab2a8578b1f7afc52c7e323c94639d11a174719aeb)

Account onboarding response:
{'account': {'accountNumber': '0',
             'createdAt': '2022-01-07T19:04:56.976Z',
             'equity': '0.000000',
             'freeCollateral': '0.000000',
             'id': 'e85ed1d9-e1e9-5516-b823-d98b11d3ca50',
             'openPositions': {},
             'pendingDeposits': '0.000000',
             'pendingWithdrawals': '0.000000',
             'positionId': '57330',
             'quoteBalance': '0.000000',
             'starkKey': '02e0e562caf451a1c91f9adc57117d6e1a00dd039f6990e8db2b394584db6bdf'},
 'apiKey': {'key': '99e1855f-ad7e-38e8-aab3-9fad0002910e',
            'passphrase': '2oCo1f2mkp4B0QjmXEHU',
      

In [19]:
priv_key1 = generate_private_key()
onboard_initial_account(
    priv_key1,
    secrets["infura_ropsten_endpoint"],
    NETWORK_ID_ROPSTEN,
    API_HOST_ROPSTEN
)

Eth Address: 0x5763265928eA73F9f2090D7E39A17E34dA1D4BB5

Eth private key: 0xcd76312715620f68cf8a650a8e404ad2484f478b029627b7d734c6c60c36aa87

STARK private key: 0x1ad026b6d19b596459d7a94c9dee38bf86977e911e458556d58bdd7b5289ef9

STARK (public x, public y): (0x5eabe831bddc373ad253fc863b39bdaa8859766d0e841f8d5ef6870c7911077,0x3bc3384191603e2f3e8e0cc514f6f577cfc70bfee27d10b59650b77b42e430d)

Account onboarding response:
{'account': {'accountNumber': '0',
             'createdAt': '2022-01-07T03:41:30.201Z',
             'equity': '0.000000',
             'freeCollateral': '0.000000',
             'id': '41eeb19c-d8c3-5fb7-aa6e-e71d74d6956f',
             'openPositions': {},
             'pendingDeposits': '0.000000',
             'pendingWithdrawals': '0.000000',
             'positionId': '57281',
             'quoteBalance': '0.000000',
             'starkKey': '05eabe831bddc373ad253fc863b39bdaa8859766d0e841f8d5ef6870c7911077'},
 'apiKey': {'key': '3f037a3a-e681-cbe0-8d2b-2cd00d0e8c7c',

In [41]:
client1 = Client(
    network_id=NETWORK_ID_ROPSTEN,
    host=API_HOST_ROPSTEN,
    default_ethereum_address="0x5763265928eA73F9f2090D7E39A17E34dA1D4BB5",
    api_key_credentials={
        'key': '3f037a3a-e681-cbe0-8d2b-2cd00d0e8c7c',
        'passphrase': '6smtWg8GnBK3T8Zp_0VT',
        'secret': 'vhMDpU5jGevy8-eY16eVnwF2Wb7zBZqdVLL-RZx-'
    },
    
)

In [43]:
token_faucet_response = client1.private.request_testnet_tokens(); token_faucet_response.data

{'transfer': {'id': '8a3a5c6b-d9d1-591a-b383-3ca98006bfeb',
  'type': 'TRANSFER_OUT',
  'debitAsset': 'USDC',
  'creditAsset': 'USDC',
  'debitAmount': '2000',
  'creditAmount': '0',
  'transactionHash': None,
  'status': 'PENDING',
  'createdAt': '2022-01-07T04:15:45.909Z',
  'confirmedAt': None,
  'clientId': '7586c569be6ab9c7',
  'fromAddress': None,
  'toAddress': None}}

In [35]:
# token_faucet_response.data

In [38]:
# client1.private.get_account().data